# 모듈 선언 및 URL 경로 지정

In [1]:
from selenium import webdriver
import time
import pandas as pd
import numpy as np
from tqdm import tqdm

url_base = 'https://www.koreabaseball.com/Record/Team/{category}'
category_list = ['Hitter/Basic1.aspx', 'Hitter/Basic2.aspx', 'Pitcher/Basic1.aspx', 'Pitcher/Basic2.aspx',
                 'Defense/Basic.aspx', 'Runner/Basic.aspx']
category_name_list = ['Hitter_1', 'Hitter_2', 'Pitcher_1', 'Pitcher_2', 'Defense', 'Runner']

driver = webdriver.Chrome('C:/chrome_driver/85/chromedriver.exe')
driver.implicitly_wait(3)

# 연도 클릭 함수 선언

In [2]:
def year_click(year):
    driver.find_element_by_class_name('select03').click()
    xpath_format = '//option[@value="{year}"]'
    driver.find_element_by_xpath(xpath_format.format(year=year)).click()

# kBO 전체 데이터 크롤링

In [3]:
for category in category_list:
    url = url_base.format(category = category) # 타자, 투수, 수비, 주루 데이터 선택
    driver.get(url)
    
    year_list = [x.strip() for x in driver.find_element_by_class_name('select03').text.split('\n') if len(x)>1] # 연도 리스트 추출
    
    for year in year_list:
        year_click(str(year)) # 각 연도 클릭
        time.sleep(0.1) # 대기
        
        temp_list = [x.split(' ') for x in driver.find_element_by_tag_name('table').text.split('\n')] # 데이터리스트 생성
                
        for i in range(len(temp_list)):
            for j in range(len(temp_list[i])):
                if '합계'==str(temp_list[i][j]): # 마지막 행 삭제
                    temp_list.pop(i)
                    break
                if '/' in str(temp_list[i][j]): # 이닝 표기문제 해결
                    temp_list[i].pop(j)
                    break
                
        temp_data = pd.DataFrame(temp_list[1:], columns=temp_list[0]) # 데이터리스트 --> 데이터프레임
        temp_data.to_csv('1. Original Datasets/'+str(year)+'_'+str(category_name_list[category_list.index(category)])+'_data.csv',
                         index=False, encoding='cp949') # 데이터프레임 저장

In [4]:
driver.close()